# Test and Benchmark

In [1]:
using Revise
using VCFTools
using LinearAlgebra
using MendelImpute
using GeneticVariation
using Random
using StatsBase
using CodecZlib
using ProgressMeter
using JLD2, FileIO, JLSO
using BenchmarkTools
using GroupSlices

┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1273


## Haplotyping timing varying number of haplotypes

In [15]:
Random.seed!(123)
ns = [100, 1000, 10000] # number of individuals
ps = [100, 1000, 10000]    # number of typed SNPs
ds = [100, 1000, 10000]  # number of reference haplotypes
for n in ns, p in ps, d in ds
    H = convert(Matrix{Float32}, rand(0:1, p, d))
    X = convert(Matrix{Float32}, rand(0:2, p, n))
    M = Transpose(H) * H
    for j in 1:d, i in 1:(j - 1) # off-diagonal
        M[i, j] = 2M[i, j] + M[i, i] + M[j, j]
    end
    for j in 1:d # diagonal
        M[j, j] *= 4
    end
    N = Transpose(X) * H
    for I in eachindex(N)
        N[I] *= 2
    end
    happair  = zeros(Int, n), zeros(Int, n)
    hapscore = zeros(eltype(N), n)
    t = @elapsed haplopair!(happair, hapscore, M, N)
    println("n = $n, p = $p, d = $d time = $t")
end

n = 100, p = 100, d = 100 time = 0.000456208
n = 100, p = 100, d = 1000 time = 0.038313572
n = 100, p = 100, d = 10000 time = 3.35376975
n = 100, p = 1000, d = 100 time = 0.0003795
n = 100, p = 1000, d = 1000 time = 0.038590982
n = 100, p = 1000, d = 10000 time = 3.374475433
n = 100, p = 10000, d = 100 time = 0.000484978
n = 100, p = 10000, d = 1000 time = 0.03808498
n = 100, p = 10000, d = 10000 time = 3.397484089
n = 1000, p = 100, d = 100 time = 0.004034878
n = 1000, p = 100, d = 1000 time = 0.3387065
n = 1000, p = 100, d = 10000 time = 34.682602842
n = 1000, p = 1000, d = 100 time = 0.004231036
n = 1000, p = 1000, d = 1000 time = 0.375277053
n = 1000, p = 1000, d = 10000 time = 36.049891365
n = 1000, p = 10000, d = 100 time = 0.003922379
n = 1000, p = 10000, d = 1000 time = 0.403318508
n = 1000, p = 10000, d = 10000 time = 35.155680319
n = 10000, p = 100, d = 100 time = 0.040836424
n = 10000, p = 100, d = 1000 time = 3.727595042
n = 10000, p = 100, d = 10000 time = 355.668273795
n 

In [ ]:
Random.seed!(123)
ns = [100, 1000]         # number of individuals
ps = 100                 # number of typed SNPs
ds = [100, 1000, 10000]  # number of reference haplotypes
for n in ns, d in ds
    H = convert(Matrix{Float32}, rand(0:1, p, d))
    X = convert(Matrix{Float32}, rand(0:2, p, n))
    M = Transpose(H) * H
    for j in 1:d, i in 1:(j - 1) # off-diagonal
        M[i, j] = 2M[i, j] + M[i, i] + M[j, j]
    end
    for j in 1:d # diagonal
        M[j, j] *= 4
    end
    N = Transpose(X) * H
    for I in eachindex(N)
        N[I] *= 2
    end
    happair  = zeros(Int, n), zeros(Int, n)
    hapscore = zeros(eltype(N), n)
    t1 = @elapsed haplopair!(happair, hapscore, M, N)
    t2 = @elapsed haplopair2!(happair, hapscore, M, N)
    println("n = $n, p = $p, d = $d time = $t")
end

In [46]:
# note for future self: the UpperTriangular matrix seems to be allocating. Getting rid of it
# probably makes the resulting code the same as haplopair!
function haplopair2!(
    happairs::Tuple{AbstractVector, AbstractVector},
    hapmin::Vector{T},
    M::AbstractMatrix{T}, #upper triangular matrix
    N::AbstractMatrix{T},
    ons::AbstractVector{T},
    Nk::AbstractVector{T},
    storage::AbstractMatrix{T}
    ) where T <: Real

    n, d = size(N)
    fill!(hapmin, typemax(T))

    # for each individual k find minimum of M_ij - n_ki - n_kj
    for k in 1:n
        copyto!(Nk, @view(N[k, :]))
        storage .= M .- UpperTriangular(Nk .* ons') .- UpperTriangular(ons .* Nk')
        hapmin[k], index = findmin(storage)
        happairs[1][k], happairs[2][k] = index[1], index[2]
    end

    return nothing
end

haplopair2! (generic function with 1 method)

In [47]:
Random.seed!(123)
# ns = [100, 1000]         # number of individuals
# ps = 100                 # number of typed SNPs
# ds = [100, 1000, 10000]  # number of reference haplotypes

n = p = 100
d = 2000

H = convert(Matrix{Float32}, rand(0:1, p, d))
X = convert(Matrix{Float32}, rand(0:2, p, n))
M = Transpose(H) * H
for j in 1:d, i in 1:(j - 1) # off-diagonal
    M[i, j] = 2M[i, j] + M[i, i] + M[j, j]
end
for j in 1:d # diagonal
    M[j, j] *= 4
end
N = Transpose(X) * H
for I in eachindex(N)
    N[I] *= 2
end
happair1 = zeros(Int, n), zeros(Int, n)
happair2 = zeros(Int, n), zeros(Int, n)
hapscore1 = zeros(eltype(N), n)
hapscore2 = zeros(eltype(N), n)

ons = ones(Float32, d)
Nk = zeros(Float32, d)
storage = zeros(Float32, d, d)

@time haplopair!(happair1, hapscore1, M, N)
@time haplopair2!(happair2, hapscore2, M, N, ons, Nk, storage)

  0.146557 seconds (4 allocations: 160 bytes)
  3.037475 seconds (140.58 k allocations: 2.986 GiB, 12.53% gc time)


In [48]:
[hapscore1 hapscore2]

100×2 Array{Float32,2}:
 -114.0  -114.0
 -105.0  -105.0
 -127.0  -127.0
 -102.0  -102.0
  -99.0   -99.0
 -101.0  -101.0
 -105.0  -105.0
 -109.0  -109.0
 -113.0  -113.0
  -94.0   -94.0
 -118.0  -118.0
  -95.0   -95.0
 -131.0  -131.0
    ⋮          
 -133.0  -133.0
 -129.0  -129.0
 -139.0  -139.0
  -96.0   -96.0
  -90.0   -90.0
  -99.0   -99.0
  -91.0   -91.0
 -119.0  -119.0
  -96.0   -96.0
  -78.0   -78.0
  -98.0   -98.0
  -89.0   -89.0

In [49]:
happair1

([534, 84, 490, 386, 99, 1602, 1221, 29, 681, 143  …  1129, 403, 155, 903, 136, 18, 470, 255, 267, 598], [1272, 459, 1285, 1004, 1390, 1738, 1551, 550, 1466, 755  …  1130, 930, 1786, 965, 1357, 1801, 1252, 1228, 1440, 1005])

In [50]:
happair2

([534, 84, 490, 386, 99, 1602, 1221, 29, 681, 143  …  1129, 403, 155, 903, 136, 18, 470, 255, 267, 598], [1272, 459, 1285, 1004, 1390, 1738, 1551, 550, 1466, 755  …  1130, 930, 1786, 965, 1357, 1801, 1252, 1228, 1440, 1005])

In [5]:
happair

([379, 363, 75, 341, 203, 157, 31, 301, 109, 291  …  175, 27, 117, 54, 408, 235, 67, 2, 241, 167], [466, 415, 358, 447, 288, 420, 402, 388, 230, 346  …  346, 132, 410, 389, 430, 475, 363, 288, 475, 175])

In [6]:
hapscore

5000-element Array{Float32,1}:
 -374.0
 -324.0
 -334.0
 -341.0
 -298.0
 -381.0
 -393.0
 -366.0
 -357.0
 -353.0
 -348.0
 -364.0
 -349.0
    ⋮  
 -336.0
 -408.0
 -403.0
 -340.0
 -373.0
 -472.0
 -335.0
 -424.0
 -381.0
 -414.0
 -347.0
 -325.0

In [8]:
@benchmark haplopair!(happair, hapscore, M, N)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     413.777 ms (0.00% GC)
  median time:      422.663 ms (0.00% GC)
  mean time:        425.083 ms (0.00% GC)
  maximum time:     444.931 ms (0.00% GC)
  --------------
  samples:          12
  evals/sample:     1

In [10]:
using Profile
Profile.clear()
@profile haplopair!(happair, hapscore, M, N)
Profile.print(format=:flat)

┌ Warning: The profile data buffer is full; profiling probably terminated
│ before your program finished. To profile for longer runs, call
│ `Profile.init()` with a larger buffer and/or larger delay.
└ @ Profile /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/Profile/src/Profile.jl:312


 Count File                        Line Function                               
    13 ./abstractarray.jl            75 axes                                   
     9 ./abstractarray.jl            95 axes1                                  
     6 ./abstractarray.jl           899 copymutable                            
     1 ./abstractarray.jl           720 copyto!(::Array{Int64,1}, ::Core.Com...
     2 ./abstractarray.jl           722 copyto!(::Array{Int64,1}, ::Core.Com...
     9 ./abstractarray.jl           799 copyto!                                
     3 ./abstractarray.jl           803 copyto!(::Core.Compiler.IndexLinear,...
     6 ./abstractarray.jl           807 copyto!(::Core.Compiler.IndexLinear,...
     7 ./abstractarray.jl           212 eachindex                              
     2 ./abstractarray.jl           268 eachindex                              
    96 ./abstractarray.jl          1920 foreach                                
     2 ./abstractarray.jl           917 

In [11]:
@code_warntype haplopair!(X, H, M, N, happair, hapscore)

Variables
  #self#::Core.Compiler.Const(MendelImpute.haplopair!, false)
  X::Array{Float32,2}
  H::Array{Float32,2}
  M::Array{Float32,2}
  N::Array{Float32,2}
  happairs::Tuple{Array{Int64,1},Array{Int64,1}}
  hapscore::Array{Float32,1}
  t0@_8::UInt64
  val@_9::Nothing
  t0@_10::UInt64
  val@_11::Nothing
  t0@_12::UInt64
  val@_13::Nothing
  val@_14::Nothing
  p::Int64
  n::Int64
  d::Int64
  @_18::Union{Nothing, Tuple{Int64,Int64}}
  @_19::Union{Nothing, Tuple{Int64,Int64}}
  t1::Float64
  t2::Float64
  @_22::Union{Nothing, Tuple{Int64,Int64}}
  t3::Float64
  j@_24::Int64
  @_25::Union{Nothing, Tuple{Int64,Int64}}
  i@_26::Int64
  j@_27::Int64
  j@_28::Int64
  @_29::Union{Nothing, Tuple{Int64,Int64}}
  r#422::Base.OneTo{Int64}
  i#423::Int64
  n#424::Int64
  i#425::Int64
  I::Int64
  j@_35::Int64
  @_36::Union{Nothing, Tuple{Int64,Int64}}
  r#426::UnitRange{Int64}
  i#427::Int64
  n#428::Int64
  i#429::Int64
  i@_41::Int64

Body::Nothing
1 ──        Core.NewvarNode(:(val@_9))
│     